<a href="https://colab.research.google.com/github/AliAbdallah21/AI-Nutrition-Coach/blob/main/Meal_%26_Grocery_Planner_with_CrewAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install langchain-community
!pip install langchain-openai
!pip install crewai
!pip install crewai-tools
!pip install databricks-sdk
!pip install duckduckgo-search

In [1]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/3xGOgzMOv5jhRsA3A8N9fQ/leftover.py"

--2025-09-19 11:05:35--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/3xGOgzMOv5jhRsA3A8N9fQ/leftover.py
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 198.23.119.245
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|198.23.119.245|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 588 [application/x-python]
Saving to: ‘leftover.py’

leftover.py         100%[===================>]     588  --.-KB/s    in 0s      

2025-09-19 11:05:35 (188 MB/s) - ‘leftover.py’ saved [588/588]



In [3]:
import sys
sys.path.append(".")

In [4]:
from leftover import LeftoversCrew

In [5]:
import os

files = os.listdir('.')
print(files)

['.config', 'leftover.py', '__pycache__', 'sample_data']


In [6]:
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
from IPython.display import display, JSON, Markdown
from datetime import datetime
import os

### **`GroceryItem`**

Think of this as designing the shopping list template before going to the store. BaseModel gives us the foundation, Field decorates our items with clear descriptions. This structure helps both humans and AI understand grocery data consistently, and includes several key string inputs:

- **`name`** – The specific grocery item to purchase (for example, "Chicken Breast").
- **`quantity`** – How much to buy in clear measurements (for example, "2 lbs", "1 gallon").
- **`estimated_price`** – The expected cost range for budgeting (for example, "$8-12").
- **`category`** – Which store section to find it in (for example, "Meat", "Produce", "Dairy").


In [7]:
class GroceryItem(BaseModel):
    """Individual grocery item"""
    name: str = Field(description="Name of the grocery item")
    quantity: str = Field(description="Quantity needed (for example, '2 lbs', '1 gallon')")
    estimated_price: str = Field(description="Estimated price (for example, '$3-5')")
    category: str = Field(description="Store section (for example, 'Produce', 'Dairy')")

In [8]:
sample_item = GroceryItem(
    name="Chicken Breast",
    quantity="2 lbs",
    estimated_price="$8-12",
    category="Meat"
)

In [9]:
sample_item

GroceryItem(name='Chicken Breast', quantity='2 lbs', estimated_price='$8-12', category='Meat')

In [10]:
type(sample_item)

__main__.GroceryItem

In [11]:
# Display structured data
print("🛒 Sample Grocery Item Structure:")
display(JSON(sample_item.model_dump()))

🛒 Sample Grocery Item Structure:


<IPython.core.display.JSON object>

### **`MealPlan`**

It represents a complete meal with all the details needed for cooking, including:

- **`meal_name`** – The name of the dish being prepared (for example, "Chicken Stir Fry").
- **`difficulty_level`** – How challenging it is to cook ("Easy", "Medium", "Hard").
- **`servings`** – Number of people the meal will feed (integer value).
- **`researched_ingredients`** – List of ingredients found through AI research (note this is a list).


In [12]:
class MealPlan(BaseModel):
    """Simple meal plan"""
    meal_name: str = Field(description="Name of the meal")
    difficulty_level: str = Field(description="'Easy', 'Medium', 'Hard'")
    servings: int = Field(description="Number of people it serves")
    researched_ingredients: List[str] = Field(description="Ingredients found through research")

In [13]:
sample_meal = MealPlan(
    meal_name="Chicken Stir Fry",
    difficulty_level="Easy",
    servings=4,
    researched_ingredients=["chicken breast", "broccoli", "bell peppers", "garlic", "soy sauce", "rice"]
)

In [14]:
sample_meal

MealPlan(meal_name='Chicken Stir Fry', difficulty_level='Easy', servings=4, researched_ingredients=['chicken breast', 'broccoli', 'bell peppers', 'garlic', 'soy sauce', 'rice'])

In [15]:
print("\n🍽️ Sample Meal Plan Structure:")
display(JSON(sample_meal.model_dump()))


🍽️ Sample Meal Plan Structure:


<IPython.core.display.JSON object>

### **`ShoppingCategory`**

It organizes items by store layout for efficient shopping, containing:

- **`section_name`** – The store department name (for example, "Produce", "Meat & Poultry").
- **`items`** – A collection of GroceryItem objects in this section (note this is a list).
- **`estimated_total`** – The expected cost for all items in this category.


In [17]:
class ShoppingCategory(BaseModel):
    """Store section with items"""
    section_name: str = Field(description="Store section (for example, 'Produce', 'Dairy')")
    items: List[GroceryItem] = Field(description="Items in this section")
    estimated_total: str = Field(description="Estimated cost for this section")

In [18]:
sample_section = ShoppingCategory(
    section_name="Produce",
    items=[
        GroceryItem(name="Bell Peppers", quantity="3 pieces", estimated_price="$3-4", category="Produce"),
        GroceryItem(name="Onions", quantity="2 lbs", estimated_price="$2-3", category="Produce")
    ],
    estimated_total="$5-7"
)

In [19]:
sample_section

ShoppingCategory(section_name='Produce', items=[GroceryItem(name='Bell Peppers', quantity='3 pieces', estimated_price='$3-4', category='Produce'), GroceryItem(name='Onions', quantity='2 lbs', estimated_price='$2-3', category='Produce')], estimated_total='$5-7')

In [20]:
print("\n🍽️ Sample Section Plan Structure:")
display(JSON(sample_section.model_dump()))


🍽️ Sample Section Plan Structure:


<IPython.core.display.JSON object>

### **`GroceryShoppingPlan`**

It is the master plan that combines everything into a complete shopping strategy:

- **`total_budget`** – The overall spending limit for the shopping trip.
- **`meal_plans`** – Collection of meals being prepared (note this is a list).
- **`shopping_sections`** – Organized categories for store navigation (note this is a list).
- **`shopping_tips`** – Money-saving advice and practical suggestions (note this is a list).


In [21]:
class GroceryShoppingPlan(BaseModel):
    """Complete simplified shopping plan"""
    total_budget: str = Field(description="Total planned budget")
    meal_plans: List[MealPlan] = Field(description="Planned meals")
    shopping_sections: List[ShoppingCategory] = Field(description="Organized by store sections")
    shopping_tips: List[str] = Field(description="Money-saving and efficiency tips")

In [25]:
%%capture

from crewai_tools import SerperDevTool
from crewai import Agent, Task, Crew, Process
from crewai import LLM
from google.colab import userdata
api_key = userdata.get('OPENAI_API_KEY')
os.environ["SERPER_API_KEY"] = userdata.get('SERPER_API_KEY')

# Initialize LLM
llm = LLM(
    model = "gpt-4o",
    api_key=api_key
)

#### Meal and Grocery Planning Workflow Overview

![image (3).png](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/0crjoLc3Yn0JefQoJVUarA/image%20-3-.png)


In [26]:
meal_planner = Agent(
    role="Meal Planner & Recipe Researcher",
    goal="Search for optimal recipes and create detailed meal plans",
    backstory="A skilled meal planner who researches the best recipes online, considering dietary needs, cooking skill levels, and budget constraints.",
    tools=[SerperDevTool()],
    llm=llm,
    verbose=False
)

In [27]:
meal_planning_task = Task(
    description=(
        "Search for the best '{meal_name}' recipe for {servings} people within a {budget} budget. "
        "Consider dietary restrictions: {dietary_restrictions} and cooking skill level: {cooking_skill}. "
        "Find recipes that match the skill level and provide complete ingredient lists with quantities."
    ),
    expected_output="A detailed meal plan with researched ingredients, quantities, and cooking instructions appropriate for the skill level.",
    agent=meal_planner,
    output_pydantic=MealPlan,
    output_file="meals.json"
)

In [28]:
from crewai import Crew, Process

meal_planner_crew = Crew(
    agents=[meal_planner],
    tasks=[meal_planning_task],
    process=Process.sequential,  # Ensures tasks are executed in order
    verbose=True
)

meal_planner_result = meal_planner_crew.kickoff(
    inputs={
        "meal_name": "Chicken Stir Fry",
        "servings": 4,
        "budget": "$25",
        "dietary_restrictions": ["no nuts"],
        "cooking_skill": "beginner"
    }
)
print("✅ Single meal planning completed!")
print("📋 Single Meal Results:")
print(meal_planner_result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: a7e24f41-7c6f-41d1-bb38-b91998c5a95c                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Task: Search for the best 'Chicken Stir Fry' recipe for 4 people within a $25 budget. Consider dietary         │
│  restrictions: ['no nuts'] and cooking skill level: beginner. Find recipes that match the skill level and       │
│  provide complete ingredient lists with quantities.                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Thought: Thought: I need to find a Chicken Stir Fry recipe that fits a beginner skill level, serves 4 people,  │
│  costs under $25, and contains no nuts.                                                                         │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"easy chicken stir fry recipe for 4 people under $25 no nuts\"}"                          │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "meal_name": "Fast & Easy Chicken Stir Fry with Vegetables",                                                 │
│    "difficulty_level": "Beginner",                                                                              │
│    "servings": 4,                                                                                               │
│    "researched_ingredients": [                                                                                  │
│      "1 lb boneless skinless chicken breasts, thinly sliced",                                                   │
│      "2 tablespoons vegetable or olive oil",                                                                    │
│      "1 cup broccoli florets",                                                                                  │
│      "1 red bell pepper, thinly sliced",                                                                        │
│      "2 carrots, thinly sliced",                                                                                │
│      "3 cloves garlic, minced",                                                                                 │
│      "1 tablespoon fresh ginger, minced",                                                                       │
│      "1/4 cup soy sauce (low sodium)",                                                                          │
│      "2 tablespoons oyster sauce",                                                                              │
│      "1 tablespoon cornstarch",                                                                                 │
│      "1/3 cup water",                                                                                           │
│      "Salt and pepper to taste",                                                                                │
│      "Serve with steamed rice (not included in price)"                                                          │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 101f8678-c3ee-4789-9565-830d33aeda0d                                                                     │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: a7e24f41-7c6f-41d1-bb38-b91998c5a95c                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "meal_name": "Fast & Easy Chicken Stir Fry with Vegetables",                                                 │
│    "difficulty_level": "Beginner",                                                                              │
│    "servings": 4,                                                                                               │
│    "researched_ingredients": [                                                                                  │
│      "1 lb boneless skinless chicken breasts, thinly sliced",                                                   │
│      "2 tablespoons vegetable or olive oil",                                                                    │
│      "1 cup broccoli florets",                                                                                  │
│      "1 red bell pepper, thinly sliced",                                                                        │
│      "2 carrots, thinly sliced",                                                                                │
│      "3 cloves garlic, minced",                                                                                 │
│      "1 tablespoon fresh ginger, minced",                                                                       │
│      "1/4 cup soy sauce (low sodium)",                                                                          │
│      "2 tablespoons oyster sauce",                                                                              │
│      "1 tablespoon cornstarch",                                                                                 │
│      "1/3 cup water",                                                                                           │
│      "Salt and pepper to taste",                                                                                │
│      "Serve with steamed rice (not included in price)"                                                          │
│    ]                                                                                                            │
│  }                                                                                                              │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

✅ Single meal planning completed!
📋 Single Meal Results:
meal_name='Fast & Easy Chicken Stir Fry with Vegetables' difficulty_level='Beginner' servings=4 researched_ingredients=['1 lb boneless skinless chicken breasts, thinly sliced', '2 tablespoons vegetable or olive oil', '1 cup broccoli florets', '1 red bell pepper, thinly sliced', '2 carrots, thinly sliced', '3 cloves garlic, minced', '1 tablespoon fresh ginger, minced', '1/4 cup soy sauce (low sodium)', '2 tablespoons oyster sauce', '1 tablespoon cornstarch', '1/3 cup water', 'Salt and pepper to taste', 'Serve with steamed rice (not included in price)']


### Creating Our Shopping Organization Agent

Now that we have a meal planner that can research recipes, we need a second specialist to transform those meal plans into organized shopping lists. This is where our **Shopping Organizer** agent comes in.

**Why do we need a separate shopping organizer?**
- **Specialization**: While the meal planner focuses on recipes and cooking, the shopping organizer specializes in store logistics and efficient shopping
- **Store Knowledge**: This agent understands how grocery stores are organized (produce, dairy, meat sections) and can group items accordingly
- **Quantity Management**: It can calculate proper quantities for different serving sizes and ensure nothing is forgotten

**Key Features of Our Shopping Organizer:**
- **No External Tools**: Unlike the meal planner, this agent doesn't need web search - it works with the meal plan data internally
- **Store Section Expertise**: Groups items by where you'll find them in the store (produce, dairy, meat, pantry)
- **Quantity Calculation**: Ensures proper amounts for the specified number of servings
- **Budget Awareness**: Considers the overall budget when organizing the shopping list


In [29]:
shopping_organizer = Agent(
    role="Shopping Organizer",
    goal="Organize grocery lists by store sections efficiently",
    backstory="An experienced shopper who knows how to organize lists for quick store trips and considers dietary restrictions.",
    tools=[],
    llm=llm,
    verbose=False
)

In [30]:
shopping_task = Task(
    description=(
        "Organize the ingredients from the '{meal_name}' meal plan into a grocery shopping list. "
        "Group items by store sections and estimate quantities for {servings} people. "
        "Consider dietary restrictions: {dietary_restrictions} and cooking skill: {cooking_skill}. "
        "Stay within budget: {budget}."
    ),
    expected_output="An organized shopping list grouped by store sections with quantities and prices.",
    agent=shopping_organizer,
    context=[meal_planning_task],
    output_pydantic=GroceryShoppingPlan,
    output_file="shopping_list.json"
)

In [31]:
two_agent_grocery_crew = Crew(
    agents=[meal_planner, shopping_organizer],  # Both agents
    tasks=[meal_planning_task, shopping_task],   # Both tasks
    process=Process.sequential,
    verbose=True
)

# Run the complete crew (this will do BOTH meal planning AND shopping)
shopping_result = two_agent_grocery_crew.kickoff(
    inputs={
        "meal_name": "Chicken Stir Fry",
        "servings": 4,
        "budget": "$25",
        "dietary_restrictions": ["no nuts"],
        "cooking_skill": "beginner"
    }
)

# Print the shopping results
print("✅ Complete meal planning + shopping completed!")
print("📋 Shopping Results:")
print(shopping_result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 1a2e9705-71cb-443c-b934-d5c4d500e5e0                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Task: Search for the best 'Chicken Stir Fry' recipe for 4 people within a $25 budget. Consider dietary         │
│  restrictions: ['no nuts'] and cooking skill level: beginner. Find recipes that match the skill level and       │
│  provide complete ingredient lists with quantities.                                                             │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Thought: Thought: I need to search for a chicken stir fry recipe that serves 4, fits a $25 budget, excludes    │
│  nuts, and is suitable for a beginner's cooking skill level.                                                    │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"best beginner chicken stir fry recipe without nuts for 4 people under 25 dollars\"}"     │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Shopping Organizer                                                                                      │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  {                                                                                                              │
│    "total_budget": "$25",                                                                                       │
│    "meal_plans": [{                                                                                             │
│      "meal_name": "Fast & Easy Chicken Stir Fry with Vegetables",                                               │
│      "difficulty_level": "Beginner",                                                                            │
│      "servings": 4,                                                                                             │
│      "researched_ingredients": [                                                                                │
│        "1 lb boneless skinless chicken thighs, trimmed and cut into bite-size pieces",                          │
│        "1 tablespoon soy sauce or gluten-free tamari",                                                          │
│        "1 tablespoon cornstarch",                                                                               │
│        "3 tablespoons cooking oil, divided",                                                                    │
│        "2 cloves garlic, minced",                                                                               │
│        "1 tablespoon ginger, minced",                                                                           │
│        "5-6 cups mixed vegetables (such as bell peppers, broccoli, and carrots)",                               │
│        "3 tablespoons soy sauce or gluten-free tamari",                                                         │
│        "1 tablespoon rice vinegar",                                                                             │
│        "1 teaspoon sesame oil (optional)"                                                                       │
│      ]                                                                                                          │
│    }],                                                                                                          │
│    "shopping_sections": [{                                                                                      │
│      "section_name": "Meat",                                                                                    │
│      "items": [{                                                                                                │
│        "name": "Chicken Thighs",                                                                                │
│        "quantity": "1 lb",                                                                                      │
│        "estimated_price": "$4.00",                                                                              │
│        "category": "poultry"                                                                                    │
│      }],                                                                                                        │
│      "estimated_total": "$4.00"                                                                                 │
│    }, {                                                                                                         │
│      "section_name": "Produce",                        

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 0a5eba0d-c7ca-4419-83e1-d788b08153c6                                                                     │
│  Agent: Shopping Organizer                                                                                      │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 1a2e9705-71cb-443c-b934-d5c4d500e5e0                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: {                                                                                                │
│    "total_budget": "$25",                                                                                       │
│    "meal_plans": [{                                                                                             │
│      "meal_name": "Fast & Easy Chicken Stir Fry with Vegetables",                                               │
│      "difficulty_level": "Beginner",                                                                            │
│      "servings": 4,                                                                                             │
│      "researched_ingredients": [                                                                                │
│        "1 lb boneless skinless chicken thighs, trimmed and cut into bite-size pieces",                          │
│        "1 tablespoon soy sauce or gluten-free tamari",                                                          │
│        "1 tablespoon cornstarch",                                                                               │
│        "3 tablespoons cooking oil, divided",                                                                    │
│        "2 cloves garlic, minced",                                                                               │
│        "1 tablespoon ginger, minced",                                                                           │
│        "5-6 cups mixed vegetables (such as bell peppers, broccoli, and carrots)",                               │
│        "3 tablespoons soy sauce or gluten-free tamari",                                                         │
│        "1 tablespoon rice vinegar",                                                                             │
│        "1 teaspoon sesame oil (optional)"                                                                       │
│      ]                                                                                                          │
│    }],                                                                                                          │
│    "shopping_sections": [{                                                                                      │
│      "section_name": "Meat",                                                                                    │
│      "items": [{                                                                                                │
│        "name": "Chicken Thighs",                                                                                │
│        "quantity": "1 lb",                                                                                      │
│        "estimated_price": "$4.00",                                                                              │
│        "category": "poultry"                                                                                    │
│      }],                                                                                                        │
│      "estimated_total": "$4.00"                                                                                 │
│    }, {                                               

✅ Complete meal planning + shopping completed!
📋 Shopping Results:
total_budget='$25' meal_plans=[MealPlan(meal_name='Fast & Easy Chicken Stir Fry with Vegetables', difficulty_level='Beginner', servings=4, researched_ingredients=['1 lb boneless skinless chicken thighs, trimmed and cut into bite-size pieces', '1 tablespoon soy sauce or gluten-free tamari', '1 tablespoon cornstarch', '3 tablespoons cooking oil, divided', '2 cloves garlic, minced', '1 tablespoon ginger, minced', '5-6 cups mixed vegetables (such as bell peppers, broccoli, and carrots)', '3 tablespoons soy sauce or gluten-free tamari', '1 tablespoon rice vinegar', '1 teaspoon sesame oil (optional)'])] shopping_sections=[ShoppingCategory(section_name='Meat', items=[GroceryItem(name='Chicken Thighs', quantity='1 lb', estimated_price='$4.00', category='poultry')], estimated_total='$4.00'), ShoppingCategory(section_name='Produce', items=[GroceryItem(name='Garlic', quantity='1 bulb', estimated_price='$0.50', category='spice'), G

### Adding Financial Intelligence with Budget Advisor Agent

Our meal planning and shopping organization is great, but we need someone to watch the budget and provide money-saving advice. Enter our **Budget Advisor** - the financial expert of our grocery team.

**Why Add a Budget Advisor?**
- **Cost Control**: Ensures the shopping plan stays within budget limits
- **Price Research**: Uses web search to find current prices and deals
- **Smart Alternatives**: Suggests cheaper ingredient substitutions when needed
- **Money-Saving Tips**: Provides practical advice for reducing grocery costs

**Budget Advisor's Unique Features:**
- **Web Search Access**: Uses `SerperDevTool()` to research current prices and deals
- **Context Awareness**: Has access to both meal planning and shopping organization results
- **Practical Focus**: Provides actionable advice rather than just price estimates


In [32]:
budget_advisor = Agent(
    role="Budget Advisor",
    goal="Provide cost estimates and money-saving tips",
    backstory="A budget-conscious shopper who helps families save money on groceries while respecting dietary needs.",
    tools=[SerperDevTool()],
    llm=llm,
    verbose=False
)

In [33]:
budget_task = Task(
    description=(
        "Analyze the shopping plan for '{meal_name}' serving {servings} people. "
        "Ensure total cost stays within {budget}. Consider dietary restrictions: {dietary_restrictions}. "
        "Provide practical money-saving tips and alternative ingredients if needed to meet budget."
    ),
    expected_output="A complete shopping guide with detailed prices, budget analysis, and money-saving tips.",
    agent=budget_advisor,
    context=[meal_planning_task, shopping_task],
    output_file="shopping_guide.md"
)

###  Using YAML with CrewAI - Food Leftover Agent and Task

####  What is YAML?

**YAML** (YAML Ain’t Markup Language) is a lightweight, human-readable format for structuring data. It is widely used in configuration files because it:

- Is easy to read and write.
- Supports nested data structures (for example, lists, dictionaries).
- Keeps logic separate from code, ideal for configuration.

####  Why Use YAML with CrewAI?

CrewAI supports YAML-based configuration for defining:
- Agents
- Tasks
- Crew processes

This allows you to:
- Quickly iterate on agent/task logic without modifying Python code.
- Enable team collaboration (even with non-developers).
- Prepare workflows for production deployment.

#### What Can You Define in YAML?

In YAML you can define te following:

- **Agents**: Roles, goals, backstories, tools, verbosity, and more
- **Tasks**: Task descriptions, expected outputs, assigned agents, and dependencies
- **Crew**: The execution flow (for example, `sequential`, `hierarchical`)

#### YAML Example: Meal Planning Crew

#### `agents.yaml`
![image (1).png](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/K-z4IjZnpV2pnhiAGpSvmw/image%20-1-.png)

#### `tasks.yaml`
![image (2).png](https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/HJ7axigzuQu8tdRR4UQMCg/image%20-2-.png)


In [34]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/-5NXnHGkX1WG81tqE8kSTg/agents.yaml"

--2025-09-19 11:34:43--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/-5NXnHGkX1WG81tqE8kSTg/agents.yaml
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 337 [text/x-yaml]
Saving to: ‘agents.yaml’

agents.yaml         100%[===================>]     337  --.-KB/s    in 0s      

2025-09-19 11:34:43 (57.6 MB/s) - ‘agents.yaml’ saved [337/337]



In [35]:
!wget "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/7fhq1KPxi7cuqjOCBgcmXQ/tasks.yaml"

--2025-09-19 11:34:46--  https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/7fhq1KPxi7cuqjOCBgcmXQ/tasks.yaml
Resolving cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)... 169.45.118.108
Connecting to cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud (cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud)|169.45.118.108|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 517 [text/x-yaml]
Saving to: ‘tasks.yaml’

tasks.yaml          100%[===================>]     517  --.-KB/s    in 0s      

2025-09-19 11:34:47 (242 MB/s) - ‘tasks.yaml’ saved [517/517]



In [36]:
%mkdir -p config
%mv agents.yaml  config/agents.yaml
%mv tasks.yaml   config/tasks.yaml

### Using CrewBase and Decorators with CrewAI

#### What is `@CrewBase`?
In CrewAI, `@CrewBase` is a Python class decorator that automates the collection and wiring of your agents and tasks — especially when you're organizing your crew logic in Python, YAML or hybrid config (YAML + code). Basically, when we have the following:

```python
from crewai.project import CrewBase
from crewai.project.annotations import agent, task

@CrewBase
class myCrew:
    ...
```
It tells CrewAI that: This class defines the agents, tasks, and crew I want to run. Please find them automatically and wire everything together.

You might think that we already defined separate Agent, Task and Crew why do we need CrewBase? We need it because this can get repetitive and hard to manage as your app grows. That’s where `@CrewBase` shines.

---

#### Why use `@CrewBase` + decorators?
- **Auto-discovers** methods decorated with @agent, @task, and @crew inside the class
- **Loads config** automatically from YAML (if present)
- Keeps everything **modular**, **scalable**, and **clean** for production setups

---

#### Decorators overview
- CrewAI provides several decorators that are used to mark methods within your crew class for special handling.
| Decorator           | Marks…                                            |
| :------------------ | :----------------------------------------------- |
| `@CrewBase`         | The class that holds your agents & tasks         |
| `@agent`            | A method that returns an `Agent` object          |
| `@task`             | A method that returns a `Task` object            |
| `@crew`             | (Optional) A method that returns a `Crew` object |
| `@before_kickoff`   | (Optional) Runs once **before** the crew starts  |
| `@after_kickoff`    | (Optional) Runs once **after** the crew finishes |

> The **CrewBase class**, along with these decorators, automates the collection of agents and tasks, reducing the need for manual management.


In [37]:
from leftover import LeftoversCrew

leftovers_cb = LeftoversCrew(llm=llm)
yaml_leftover_manager = leftovers_cb.leftover_manager()
yaml_leftover_task    = leftovers_cb.leftover_task()

In [38]:
summary_agent = Agent(
    role="Report Compiler",
    goal="Compile comprehensive meal planning reports from all team outputs",
    backstory="A skilled coordinator who organizes information from multiple specialists into comprehensive, easy-to-follow reports.",
    tools=[],
    llm=llm,
    verbose=False
)

In [39]:
summary_task = Task(
    description=(
        "Compile a comprehensive meal planning report that includes:\n"
        "1. The complete recipe and cooking instructions from the meal planner\n"
        "2. The organized shopping list with prices from the shopping organizer\n"
        "3. The budget analysis and money-saving tips from the budget advisor\n"
        "4. The leftover management suggestions from the waste reduction specialist\n"
        "Format this as a complete, user-friendly meal planning guide."
    ),
    expected_output="A comprehensive meal planning guide that combines all team outputs into one cohesive report.",
    agent=summary_agent,
    context=[meal_planning_task, shopping_task, budget_task, yaml_leftover_task],
)

### Assembling Our Complete Grocery Planning Team

Next, we bring together all five specialists into one powerful grocery planning crew. This represents our complete end-to-end solution from meal idea to comprehensive shopping guide with zero-waste strategies.

**Our Five-Agent Team:**
1. **Meal Planner**: Researches recipes and creates meal plans
2. **Shopping Organizer**: Transforms meal plans into organized shopping lists  
3. **Budget Advisor**: Adds financial analysis and money-saving strategies
4. **Leftover Manager**: Minimizes food waste by suggesting creative uses for leftover ingredients
5. **Report Compiler**: Consolidates all outputs into one comprehensive, user-friendly guide


In [40]:
complete_grocery_crew = Crew(
    agents=[
        meal_planner,
        shopping_organizer,
        budget_advisor,
        yaml_leftover_manager,  # YAML-based leftover manager
        summary_agent           # New summary agent
    ],
    tasks=[
        meal_planning_task,
        shopping_task,
        budget_task,
        yaml_leftover_task,    # YAML-based leftover task
        summary_task            # New summary task
    ],
    process=Process.sequential,
    verbose=True
)

In [41]:
# Run the complete crew
complete_result = complete_grocery_crew.kickoff(
    inputs={
        "meal_name": "Chicken Stir Fry",
        "servings": 4,
        "budget": "$25",
        "dietary_restrictions": ["no nuts", "low sodium"],
        "cooking_skill": "beginner"
    }
)

print("✅ Complete meal planning with summary completed!")
print("📋 Complete Results:")
print(complete_result)

╭──────────────────────────────────────────── Crew Execution Started ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Started                                                                                         │
│  Name: crew                                                                                                     │
│  ID: 284ad0d6-db91-4be6-ab0a-a4590189a2c7                                                                       │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭─────────────────────────────────────────────── 🤖 Agent Started ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Task: Search for the best 'Chicken Stir Fry' recipe for 4 people within a $25 budget. Consider dietary         │
│  restrictions: ['no nuts', 'low sodium'] and cooking skill level: beginner. Find recipes that match the skill   │
│  level and provide complete ingredient lists with quantities.                                                   │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

Output()

╭──────────────────────────────────────────── 🔧 Agent Tool Execution ────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Meal Planner & Recipe Researcher                                                                        │
│                                                                                                                 │
│  Thought: Thought: I need to search for beginner-friendly "Chicken Stir Fry" recipes that are suitable for 4    │
│  people, under a $25 budget, and comply with the specified dietary restrictions.                                │
│                                                                                                                 │
│  Using Tool: Search the internet with Serper                                                                    │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭────────────────────────────────────────────────── Tool Input ───────────────────────────────────────────────────╮
│                                                                                                                 │
│  "{\"search_query\": \"Chicken Stir Fry recipe for 4 people beginner no nuts low sodium budget\"}"              │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

Output()

╭───────────────────────────────────────────── ✅ Agent Final Answer ─────────────────────────────────────────────╮
│                                                                                                                 │
│  Agent: Report Compiler                                                                                         │
│                                                                                                                 │
│  Final Answer:                                                                                                  │
│  # Comprehensive Meal Planning Guide for "Easy 5-star Chicken Stir Fry"                                         │
│                                                                                                                 │
│  ## Recipe Overview                                                                                             │
│  **Dish Name:** Easy 5-star Chicken Stir Fry                                                                    │
│  **Difficulty Level:** Beginner                                                                                 │
│  **Servings:** 4                                                                                                │
│                                                                                                                 │
│  ### Complete Recipe and Cooking Instructions                                                                   │
│                                                                                                                 │
│  #### Ingredients:                                                                                              │
│  - 1 lb. boneless skinless chicken breast                                                                       │
│  - 2 tablespoons cooking oil                                                                                    │
│  - 2 cups snap peas                                                                                             │
│  - 1 large bell pepper                                                                                          │
│  - 1 cup carrot sticks                                                                                          │
│  - 2 cups cooked rice                                                                                           │
│  - 3 tablespoons low-sodium soy sauce                                                                           │
│  - 1 tablespoon sesame oil                                                                                      │
│  - 2 cloves garlic, minced                                                                                      │
│  - 1 tablespoon fresh ginger, grated                                                                            │
│  - 1 tablespoon cornstarch                                                                                      │
│  - 1 tablespoon brown sugar                                                                                     │
│  - 1/4 teaspoon crushed red pepper flakes                                                                       │
│                                                                                                                 │
│  #### Cooking Instructions:                                                                                     │
│  1. Slice the chicken breast into thin strips or bite-size pieces.                                              │
│  2. Heat 1 tablespoon of the cooking oil in a large skillet over medium-high heat. Once hot, add the chicken    │
│  and stir until cooked through. Remove from skillet and set aside.                                              │
│  3. In the same skillet, add the remaining oil along with snap peas, bell pepper strips, and carrot sticks.     │
│  Stir fry the vegetables for about 5 minutes or until t

╭──────────────────────────────────────────────── Task Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Task Completed                                                                                                 │
│  Name: 9854be6d-944e-41bb-8f19-e282b99d0529                                                                     │
│  Agent: Report Compiler                                                                                         │
│  Tool Args:                                                                                                     │
│                                                                                                                 │
│                                                                                                                 │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── Crew Completion ────────────────────────────────────────────────╮
│                                                                                                                 │
│  Crew Execution Completed                                                                                       │
│  Name: crew                                                                                                     │
│  ID: 284ad0d6-db91-4be6-ab0a-a4590189a2c7                                                                       │
│  Tool Args:                                                                                                     │
│  Final Output: # Comprehensive Meal Planning Guide for "Easy 5-star Chicken Stir Fry"                           │
│                                                                                                                 │
│  ## Recipe Overview                                                                                             │
│  **Dish Name:** Easy 5-star Chicken Stir Fry                                                                    │
│  **Difficulty Level:** Beginner                                                                                 │
│  **Servings:** 4                                                                                                │
│                                                                                                                 │
│  ### Complete Recipe and Cooking Instructions                                                                   │
│                                                                                                                 │
│  #### Ingredients:                                                                                              │
│  - 1 lb. boneless skinless chicken breast                                                                       │
│  - 2 tablespoons cooking oil                                                                                    │
│  - 2 cups snap peas                                                                                             │
│  - 1 large bell pepper                                                                                          │
│  - 1 cup carrot sticks                                                                                          │
│  - 2 cups cooked rice                                                                                           │
│  - 3 tablespoons low-sodium soy sauce                                                                           │
│  - 1 tablespoon sesame oil                                                                                      │
│  - 2 cloves garlic, minced                                                                                      │
│  - 1 tablespoon fresh ginger, grated                                                                            │
│  - 1 tablespoon cornstarch                                                                                      │
│  - 1 tablespoon brown sugar                                                                                     │
│  - 1/4 teaspoon crushed red pepper flakes                                                                       │
│                                                                                                                 │
│  #### Cooking Instructions:                                                                                     │
│  1. Slice the chicken breast into thin strips or bite-size pieces.                                              │
│  2. Heat 1 tablespoon of the cooking oil in a large skillet over medium-high heat. Once hot, add the chicken    │
│  and stir until cooked through. Remove from skillet and set aside.                                              │
│  3. In the same skillet, add the remaining oil along w

✅ Complete meal planning with summary completed!
📋 Complete Results:
# Comprehensive Meal Planning Guide for "Easy 5-star Chicken Stir Fry"

## Recipe Overview
**Dish Name:** Easy 5-star Chicken Stir Fry  
**Difficulty Level:** Beginner  
**Servings:** 4  

### Complete Recipe and Cooking Instructions

#### Ingredients:
- 1 lb. boneless skinless chicken breast
- 2 tablespoons cooking oil
- 2 cups snap peas
- 1 large bell pepper
- 1 cup carrot sticks
- 2 cups cooked rice
- 3 tablespoons low-sodium soy sauce
- 1 tablespoon sesame oil
- 2 cloves garlic, minced
- 1 tablespoon fresh ginger, grated
- 1 tablespoon cornstarch
- 1 tablespoon brown sugar
- 1/4 teaspoon crushed red pepper flakes

#### Cooking Instructions:
1. Slice the chicken breast into thin strips or bite-size pieces.
2. Heat 1 tablespoon of the cooking oil in a large skillet over medium-high heat. Once hot, add the chicken and stir until cooked through. Remove from skillet and set aside.
3. In the same skillet, add the remain

### Understanding Your Complete Grocery Shopping Guide

Congratulations! You've successfully created an AI-powered grocery shopping assistant that transforms a simple meal request into a comprehensive shopping strategy.

**What You've Accomplished:**
- **Automated Recipe Research**: No more browsing endless recipes online
- **Organized Shopping Lists**: Items grouped by store sections for efficient shopping
- **Budget Management**: Cost analysis and money-saving recommendations
- **Dietary Compliance**: All suggestions respect your dietary restrictions and cooking skill level
- **Waste Reduction**: Creative leftover management with bonus recipes to maximize your grocery budget
- **Comprehensive Reporting**: All information compiled into one easy-to-follow guide

**Next Steps:**
You can now adapt this framework for any cuisine, dietary need, or budget constraint. Try different meal requests, adjust serving sizes, experiment with various dietary restrictions, or test different cooking skill levels to see how the system adapts. The five-agent team will handle everything from recipe research to waste reduction, ensuring you get maximum value from every grocery trip.
